In [3]:
!docker ps 

CONTAINER ID        IMAGE                                 COMMAND                  CREATED             STATUS              PORTS                       NAMES
77d9f8d3237e        mirantis/kubeadm-dind-cluster:v1.11   "/sbin/dind_init sys…"   8 minutes ago       Up 8 minutes        8080/tcp                    kube-node-2
f06e4cdac31f        mirantis/kubeadm-dind-cluster:v1.11   "/sbin/dind_init sys…"   8 minutes ago       Up 8 minutes        8080/tcp                    kube-node-1
e7e5aa014a4a        mirantis/kubeadm-dind-cluster:v1.11   "/sbin/dind_init sys…"   10 minutes ago      Up 10 minutes       127.0.0.1:32770->8080/tcp   kube-master


In [2]:
!kubectl get nodes

NAME          STATUS   ROLES    AGE   VERSION
kube-master   Ready    master   9m    v1.11.0
kube-node-1   Ready    <none>   7m    v1.11.0
kube-node-2   Ready    <none>   7m    v1.11.0


## kubelet

kubelet is the core agent run by systemd. 
It is the start point of a kerbernets node, either Head Node or Worker Node


In [5]:
!docker exec -i kube-master systemctl status kubelet

● kubelet.service - kubelet: The Kubernetes Node Agent
   Loaded: loaded (/lib/systemd/system/kubelet.service; enabled; vendor preset: enabled)
   Active: active (running) since Wed 2018-10-31 14:10:40 UTC; 9min ago
     Docs: http://kubernetes.io/docs/
 Main PID: 3837 (hyperkube)
    Tasks: 20 (limit: 4915)
   Memory: 53.6M
      CPU: 18.604s
   CGroup: /docker/e7e5aa014a4aae2b25eed47e768245fa339670a92e061c82549172a678c3ea0b/system.slice/kubelet.service
           └─3837 /k8s/hyperkube kubelet --kubeconfig=/etc/kubernetes/kubelet.conf --pod-manifest-path=/etc/kubernetes/manifests --allow-privileged=true --network-plugin=cni --cni-conf-dir=/etc/cni/net.d --cni-bin-dir=/opt/cni/bin --cluster-dns=10.96.0.10 --cluster-domain=cluster.local --eviction-hard=memory.available<100Mi,nodefs.available<100Mi,nodefs.inodesFree<1000 --fail-swap-on=false --bootstrap-kubeconfig=/etc/kubernetes/bootstrap-kubelet.conf --feature-gates=MountPropagation=true,DynamicKubeletConfig=true --v=4

Oct 31 14:20:23

In [6]:
!docker exec -i kube-master cat /etc/kubernetes/kubelet.conf

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUN5RENDQWJDZ0F3SUJBZ0lCQURBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwcmRXSmwKY201bGRHVnpNQjRYRFRFNE1UQXpNVEUwTURneE5Wb1hEVEk0TVRBeU9ERTBNRGd4TlZvd0ZURVRNQkVHQTFVRQpBeE1LYTNWaVpYSnVaWFJsY3pDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBTFRVCnBsV1hmTG9OQ0JNazNMNXNlNW90T3c5em5DWThSeDh2RUd3S2xJeEZmNy92Zy8zNVdaalB0aU1mYS9TNXMyOWQKa0gxWkZ0OUs2WEJJSEI3QmxVemhoSk14VEYxeVlEV2Qwd2llV1FtMlM1TC9FdEhoZlBESzdjZFBQSUtmSDBJRAphNDVLN0hEZUI1dTFqb280dUlHWDlyRmxld1J4S2VKVDJzWlZrbkhZVnFuRXRGRlZ4T3dIRkcrdUppQ0pYbU9MCkFvRm5XRUdxQ0FJNy91M01XdVZRMzVsUFVxWm1XeXVSWldkY0RobTNTa0o1cVpVZ2x1dGNRSm9JNWxrakJPT28KT1BzSFhKeWFKVDRiWHFJV1JVME4xOTJzTmpRclhOdnFBTXhLZlBkV2hKL01vRlhuTXFyQ2c0Um1ZRmFsQVg0bwpHZjM0UERWZmFsNjVsUFJTOVMwQ0F3RUFBYU1qTUNFd0RnWURWUjBQQVFIL0JBUURBZ0trTUE4R0ExVWRFd0VCCi93UUZNQU1CQWY4d0RRWUpLb1pJaHZjTkFRRUxCUUFEZ2dFQkFDSUNWVVNtTnJzSVFvZzNqbTlqTE9vd3V2eTYKQVJxZGlEekU4cXRCeXRzYkxsV2hu

In [8]:
!docker exec -i kube-master ls -alh /etc/kubernetes/manifests

total 24K
drwx------ 2 root root 4.0K Oct 31 14:10 .
drwxr-xr-x 4 root root 4.0K Oct 31 14:08 ..
-rw------- 1 root root 2.0K Oct 31 14:08 etcd.yaml
-rw------- 1 root root 3.7K Oct 31 14:09 kube-apiserver.yaml
-rw------- 1 root root 2.8K Oct 31 14:09 kube-controller-manager.yaml
-rw------- 1 root root 1.5K Oct 31 14:09 kube-scheduler.yaml


In [13]:
!docker exec -i kube-master cat /etc/kubernetes/manifests/kube-apiserver.yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    scheduler.alpha.kubernetes.io/critical-pod: ""
  creationTimestamp: null
  labels:
    component: kube-apiserver
    tier: control-plane
  name: kube-apiserver
spec:
  containers:
  - command:
    - kube-apiserver
    - --authorization-mode=Node,RBAC
    - --feature-gates=MountPropagation=true
    - --advertise-address=10.192.0.2
    - --allow-privileged=true
    - --client-ca-file=/etc/kubernetes/pki/ca.crt
    - --disable-admission-plugins=PersistentVolumeLabel
    - --enable-admission-plugins=NodeRestriction
    - --enable-bootstrap-token-auth=true
    - --etcd-cafile=/etc/kubernetes/pki/etcd/ca.crt
    - --etcd-certfile=/etc/kubernetes/pki/apiserver-etcd-client.crt
    - --etcd-keyfile=/etc/kubernetes/pki/apiserver-etcd-client.key
    - --etcd-servers=https://127.0.0.1:2379
    - --kubelet-client-certificate=/etc/kubernetes/pki/apiserver-kubelet-client.crt
    - --kubelet-client-key=/etc/kubernetes/pki/apiserver-kubelet-client.k

In [12]:
!docker exec -i kube-master ls /etc/kubernetes

admin.conf
controller-manager.conf
kubelet.conf
manifests
pki
scheduler.conf


In [14]:
!docker exec -i kube-master docker ps

CONTAINER ID        IMAGE                  COMMAND                  CREATED             STATUS              PORTS               NAMES
49e5a62e51c9        cf2bb25e5b43           "/usr/local/bin/ku..."   17 minutes ago      Up 17 minutes                           k8s_kube-proxy_kube-proxy-l56gd_kube-system_b50922e7-dd16-11e8-9e2f-66621ed27d83_1
9e7922ff8c87        k8s.gcr.io/pause:3.1   "/pause"                 17 minutes ago      Up 17 minutes                           k8s_POD_kube-proxy-l56gd_kube-system_b50922e7-dd16-11e8-9e2f-66621ed27d83_1
043abfa27000        cf2bb25e5b43           "kube-scheduler --..."   17 minutes ago      Up 17 minutes                           k8s_kube-scheduler_kube-scheduler-kube-master_default_27a9e6670b45047dfa1152fa922aff56_1
ab694ba5f5f0        b8df3b177be2           "etcd --advertise-..."   17 minutes ago      Up 17 minutes                           k8s_etcd_etcd-kube-master_kube-system_78263d83ff9d8e4fa24f4ff1b321f5b4_1
8e856f45fd56        cf2bb25e5b43 

In [15]:
!docker exec -i kube-master ps -ef

UID        PID  PPID  C STIME TTY          TIME CMD
root         1     0  0 14:07 ?        00:00:00 /sbin/dind_init systemd.setenv=CNI_PLUGIN=bridge systemd.setenv=IP_MODE=ipv4 systemd.setenv=DIND_STORAGE_DRIVER=overlay2 systemd.setenv=POD_NET_PREFIX="10.244.1" systemd.setenv=POD_NET_SIZE="24" systemd.setenv=POD_NET2_PREFIX="" systemd.setenv=POD_NET2_SIZE="" systemd.setenv=SERVICE_NET_MODE=ipv4 systemd.setenv=USE_HAIRPIN=false systemd.setenv=DNS_SVC_IP=10.96.0.10 systemd.setenv=DNS_SERVICE=kube-dns
root        17     1  0 14:07 ?        00:00:03 /lib/systemd/systemd-journald
root        49     1  0 14:07 ?        00:00:00 /bin/bash /usr/local/bin/dindnet
root       101    49  0 14:07 ?        00:00:00 socat udp4-recvfrom:53,reuseaddr,fork,bind=10.192.0.2 UDP:127.0.0.11:53
root      3837     1  2 14:10 ?        00:00:32 /k8s/hyperkube kubelet --kubeconfig=/etc/kubernetes/kubelet.conf --pod-manifest-path=/etc/kubernetes/manifests --allow-privileged=true --network-plugin=cni --cni-conf-di

In [20]:
!docker exec -i kube-master ls /etc/systemd/system

docker.service.d
docker.service.wants
getty.target.wants
multi-user.target.wants
sockets.target.wants
sysinit.target.wants


In [ ]:
!docker exec -i kube-master

In [ ]:
!docker exec -i kube-master

In [ ]:
!docker exec -i kube-master